<a href="https://colab.research.google.com/github/aghabayli/ensemble/blob/master/Soft_various_ensembling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Connect to GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
#Connect to the google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#!apt-get install python3-venv
#!python3 -m venv myenv/env_ensemble
#!source myenv/env_ensemble/bin/activate
!pip uninstall numpy==1.16.3
!pip install numpy==1.16.1
#!pip uninstall kashgari
!pip install /content/gdrive/My\ Drive/Kashgari/

In [0]:
#Import libraries
from keras.datasets import reuters
import numpy as np
from sklearn.model_selection import train_test_split

#Download the Reuters data set from Keras
(X_data, y_data), (X_test, y_test)  = reuters.load_data(path="../reuters.npz", num_words=None,skip_top=0,maxlen=None,test_split=0.2,seed=113,start_char=1,oov_char=2,index_from=3)

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.60, random_state=113)

In [0]:
print(X_train[0])
print(y_train[0])
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

[1, 37, 412, 961, 202, 10, 146, 1315, 878, 466, 750, 445, 7, 91, 6, 10, 135, 32, 767, 11, 58, 83, 59, 20, 5, 4, 2023, 4, 1427, 213, 8, 7, 93, 878, 466, 961, 202, 27, 42, 8207, 553, 360, 639, 6, 12, 1477, 11, 58, 83, 72, 20, 16, 8, 4, 1832, 4203, 135, 7, 91, 348, 5739, 10745, 11480, 202, 10, 146, 2442, 18, 6, 32, 818, 11, 58, 83, 63, 20, 22, 6906, 12, 160, 11, 160, 32, 20, 91, 34, 4, 8454, 4494, 433, 22, 10, 2580, 54, 1489, 5, 160, 59, 20, 4, 1427, 213, 8, 961, 40, 1909, 6, 136, 109, 11, 13, 4, 99, 216, 7, 117, 109, 206, 4, 91, 302, 5, 750, 18, 34, 4, 473, 499, 224, 4045, 62, 99, 2344, 7, 10950, 1427, 262, 7221, 11801, 8, 137, 2554, 57, 994, 488, 305, 2260, 7, 295, 277, 961, 281, 4, 585, 29, 36, 152, 4, 54, 216, 4, 466, 135, 5, 4046, 34, 420, 109, 11, 34, 429, 1049, 184, 4, 1832, 135, 34, 54, 420, 52, 320, 7, 252, 1600, 17, 12]
30
(3592,)
(3592,)
(5390,)
(5390,)
(2246,)
(2246,)


In [0]:
#Constructing data to available input format at Kashgari
X_train = [[str(j) for j in i ]for i in X_train ]
y_train = [ str(i) for i in y_train ]

X_val = [[str(j) for j in i ]for i in X_val ]
y_val = [ str(i) for i in y_val ]

X_test = [[str(j) for j in i ]for i in X_test ]
y_test = [ str(i) for i in y_test ]

In [0]:
print(X_train[0])
y_train[0]

['1', '37', '412', '961', '202', '10', '146', '1315', '878', '466', '750', '445', '7', '91', '6', '10', '135', '32', '767', '11', '58', '83', '59', '20', '5', '4', '2023', '4', '1427', '213', '8', '7', '93', '878', '466', '961', '202', '27', '42', '8207', '553', '360', '639', '6', '12', '1477', '11', '58', '83', '72', '20', '16', '8', '4', '1832', '4203', '135', '7', '91', '348', '5739', '10745', '11480', '202', '10', '146', '2442', '18', '6', '32', '818', '11', '58', '83', '63', '20', '22', '6906', '12', '160', '11', '160', '32', '20', '91', '34', '4', '8454', '4494', '433', '22', '10', '2580', '54', '1489', '5', '160', '59', '20', '4', '1427', '213', '8', '961', '40', '1909', '6', '136', '109', '11', '13', '4', '99', '216', '7', '117', '109', '206', '4', '91', '302', '5', '750', '18', '34', '4', '473', '499', '224', '4045', '62', '99', '2344', '7', '10950', '1427', '262', '7221', '11801', '8', '137', '2554', '57', '994', '488', '305', '2260', '7', '295', '277', '961', '281', '4', '58

'30'

In [1]:
#Import libraries to train
from kashgari.tasks.classification import CNNModel, CNNLSTMModel, BLSTMModel, DropoutBGRUModel,KMaxCNNModel 
from sklearn.metrics import accuracy_score, confusion_matrix
from numpy import dstack

Using TensorFlow backend.


In [0]:
def prediction(models, X_data, y_data = None):

  predictions = None

  #Predict for each individual model
  for model in models:
    #Soft classification prediction
    pr = model.predict(X_data, soft=True)
    
    #model.evaluate(X_data, y_data)
    
    #Stacking depth wise
    if predictions is None:
      predictions = np.array(pr)
    else:
      predictions = dstack((predictions, np.array(pr)))
  return predictions

In [0]:
#Identify individual models
cl1 = CNNModel()
cl2 = BLSTMModel()
cl3 = DropoutBGRUModel()
cl4 = KMaxCNNModel()

#List of the models
classifier = [cl1, cl2, cl3, cl4]

predictions = None
#Create a list for pre-ensemble models
models = list()


#Train n individual models
for i in range(len(classifier)):
    #Fit train data
    classifier[i].fit(X_train,y_train, multi_label=True, epochs = 3)
    #Append to list trained classifier
    models.append(classifier[i])
    #Predict soft classification on train data
    predicted = classifier[i].predict(X_train, soft=True)
    #Stack all predictions 
    if predictions is None:
        predictions = np.array(predicted)
    else:
        #Stack depth wise
        predictions = dstack((predictions, np.array(predicted)))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 450)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 450, 100)          959800    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 446, 128)          64128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 47)                3055      
Total params: 1,035,239
Trainable params: 1,035,239
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Epoch 1/3
56/56 [==============================] - 5s 86ms/step - loss: 2.7961 - acc: 0.3728
Epoch 2/3
56/56 [==============================] - 1s 18ms/step - loss: 1.8519 - acc: 0.5246
Epoch 3/3
56/56 [==============================] - 1s 19ms/step - loss: 1.5497 - acc: 0.6275
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 450)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 450, 100)          959800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               731136    
_________________________________________________________________
dense_3 (Dense)              (None, 47)                24111     
Total params: 1,715,047
Trainable params: 1,715,047
Non-trainable params: 0
___________________________________________________

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 450)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 450, 100)     959800      input_3[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 450, 100)     0           embedding_3[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 450, 128)     63360       spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
dropout_1 

In [0]:
#CNN model from Kashgari
classifier = CNNModel()

#Predicting from pre-ensemble models   
val_predictions = prediction(models, X_val)

#Reshape to 2D array     
X_predict =  val_predictions.reshape(val_predictions.shape[0], val_predictions.shape[1]*val_predictions.shape[2])

#Round probabilities to two digits
X_predict = np.around(X_predict, 2)

#Structure predictions to Kashgari input format
X_predict = [[str(j) for j in i ]for i in X_predict ]

#Fit predicted probabilities as the input to the model
classifier.fit(X_predict, y_val, epochs=5)

#Predict on final model
ensemble_pr = classifier.predict(X_predict)

#print(confusion_matrix(y_val, ensemble_pr))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 188)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 188, 100)          10500     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 184, 128)          64128     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_13 (Dense)             (None, 47)                3055      
Total params: 85,939
Trainable params: 85,939
Non-trainable params: 0
_________________________________________________________________
Epoch 

In [0]:
#Predicting from pre-ensemble models
test_predictions = prediction(models, X_test)
    
#Reshape to 2D    
X_predict =  test_predictions.reshape(test_predictions.shape[0], test_predictions.shape[1]*test_predictions.shape[2])

#Round probabilities to two digits
X_predict = np.around(X_predict, 2)

#Predict on final model
X_predict = [[str(j) for j in i ]for i in X_predict ]

#Final accuracy
print("Ensemble accuracy:")
print(accuracy_score(classifier.predict(X_predict), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        12
           1     0.3447    0.6762    0.4566       105
          10     0.0000    0.0000    0.0000        30
          11     0.5625    0.3253    0.4122        83
          12     0.0000    0.0000    0.0000        13
          13     0.0000    0.0000    0.0000        37
          14     0.0000    0.0000    0.0000         2
          15     0.0000    0.0000    0.0000         9
          16     0.4780    0.7677    0.5891        99
          17     0.0000    0.0000    0.0000        12
          18     0.0000    0.0000    0.0000        20
          19     0.3353    0.8346    0.4784       133
           2     0.0000    0.0000    0.0000        20
          20     0.0000    0.0000    0.0000        70
          21     0.0000    0.0000    0.0000        27
          22     0.0000    0.0000    0.0000         7
          23     0.0000    0.0000    0.0000        12
          24     0.0000    

0.6366874443455031